In [ ]:
#https://www.datamanim.com/dataset/99_pandas/pandasMain.html
#print(dir(DataFrame)) #print(help(sklearn))

#1. 유튜브 인기동영상 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv",index_col=0)

#1. 인기동영상 제작횟수가 많은 채널 상위 10개명을 출력하라 (날짜기준, 중복포함)
#isin: 포함여부
list(df.loc[df.channelId.isin(df.channelId.value_counts().head(10).index)].channelTitle.unique())

#2. dislikes수가 like 수보다 높은 동영상을 제작한 채널을 모두 출력하라
list(df.loc[df.likes < df.dislikes].channelTitle.unique())

#3. 채널명을 한번이라도 바꾼 채널의 갯수
#drop_duplicates(): 중복일 때. 하나만 남기고 나머지 삭제.
chanel = df[['channelTitle', 'channelId']].drop_duplicates().channelId.value_counts()
len(chanel[chanel>1])

#★4. 일요일에 인기있었던 영상들중 가장 많은 영상 종류(categoryId)는 무엇인가?
#df.date2.dt.day_name() = 요일
df['trending_date2'] = pd.to_datetime(df['trending_date2'])
df.loc[df.trending_date2.dt.day_name() == 'Sunday'].categoryId.value_counts().index[0]

#★5. 각 요일별 인기 영상들의 categoryId는 각각 몇개 씩인지 하나의 데이터 프레임으로 표현하라
#as_index = False : 2번째 줄이 인덱스가 되는 것
data = df.groupby([df.trending_date2.dt.day_name(), 'categoryId'], as_index=False).size()
data.pivot(index='categoryId', columns='trending_date2')

#6. viewcount대비 댓글수가 가장 높은 영상을 확인하라 (view_count값이 0인 경우는 제외한다)
#value_counts(): 빈도수에 따라 오름차순
#sort_values(by='ratio', ascending=False): 기준에 따라 오름차순
target = df.loc[df.view_count!=0]
t = target.copy()
t['ratio'] = (target.comment_count/target.view_count).dropna()
t.sort_values(by='ratio', ascending=False).iloc[0].title

#8. like 대비 dislike의 수가 가장 적은 영상은 무엇인가? (like, dislike 값이 0인경우는 제외한다)
df['ratio'] = (df.dislikes/df.likes).dropna()
target = df.loc[(df.likes != 0) & (df.dislikes != 0)]
target.sort_values(by='ratio', ascending=True).iloc[0].title

#9. 가장많은 트렌드 영상을 제작한 채널의 이름은 무엇인가? (날짜기준, 중복포함)
df.loc[df.channelId ==df.channelId.value_counts().index[0]].channelTitle.unique()[0]

#10. 20회(20일)이상 인기동영상 리스트에 포함된 동영상의 숫자는?
#title은 같아도 id가 다를 수 있어서 둘다 고려.
(df[['title', 'channelId']].value_counts() >= 20).sum()

#2. 유튜브 공범컨텐츠 동영상 데이터

In [ ]:
import pandas as pd

channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')

#11.‘ct’컬럼을 시간으로 인식하고, video 데이터의 videoname의 각 value 마다 몇개의 데이터씩 가지고 있는지
channel.ct = pd.to_datetime(channel.ct)
video.ct = pd.to_datetime(video.ct)
video.videoname.value_counts()


#12. 수집된 각 video의 가장 최신화 된 날짜의 viewcount값
answer = video.sort_values(['videoname','ct']).drop_duplicates('videoname', keep='last')
answer[['viewcnt', 'videoname', 'ct']].reset_index(drop=True)


#★13. Channel 데이터중 2021-10-03일 이후 각 채널의 처음 기록 됐던 구독자 수(subcnt)를 출력
#drop_duplicates('중복 기준', keep='남길 것 기준')
#reset_index(drop=원래 index 어떻게 할지)
#pd.to_datetime('날짜')
answer = channel[channel.ct >= pd.to_datetime('2021-10-03')].sort_values(['ct', 'channelname']).drop_duplicates('channelname', keep='first')
answer[['channelname','subcnt']].reset_index(drop=True)


#★14. 각채널의 2021-10-03 03:00:00 ~ 2021-11-01 15:00:00 까지 구독자수 (subcnt) 의 증가량
#channel.ct.dt.strftime('%Y-%m-%d %H') =='2021-11-01 15'
end = channel.loc[channel.ct.dt.strftime('%Y-%m-%d %H') =='2021-11-01 15']
start = channel.loc[channel.ct.dt.strftime('%Y-%m-%d %H') =='2021-10-03 03']

end_df = end[['channelname','subcnt']].reset_index(drop=True)
start_df = start[['channelname','subcnt']].reset_index(drop=True)

end_df.columns = ['channelname','end_sub']
start_df.columns = ['channelname','start_sub']

tt = pd.merge(start_df,end_df)
tt['del'] = tt['end_sub'] - tt['start_sub']
tt[['channelname','del']]


#★15. 공범 EP1의 비디오정보 데이터중 수집간격이 5분 이하, 20분이상인 데이터 구간( 해당 시점 전,후) 의 시각을 모두 출력
#ep_one.ct.diff(1)>=datetime.timedelta(minutes=20)
import datetime
ep_one = video.loc[video.videoname.str.contains('1')].sort_values('ct').reset_index(drop=True)
ep_one[(ep_one.ct.diff(1)>=datetime.timedelta(minutes=20)) | (ep_one.ct.diff(1) <=datetime.timedelta(minutes=5))]
ep_one[ep_one.index.isin([720,721,722,723,1635,1636,1637])]


#16. 각 에피소드의 시작날짜(년-월-일)를 에피소드 이름과 묶어 데이터 프레임
start_date = video.sort_values(['ct', 'videoname']).drop_duplicates('videoname')[['ct', 'videoname']]
start_date['ct'] = pd.to_datetime(start_date['ct'])
start_date['date'] = start_date.ct.dt.date
  #start_date['date'] = start_date.ct.dt..strftime('%Y-%m-%d')
start_date[['date', 'videoname']]


#17. 공범이 공개된 날의 21시의 viewcnt, ct, videoname 으로 구성된 데이터 프레임을 viewcnt를 내림차순으로 정렬하여 출력
video.loc[video.ct.dt.hour == 21].sort_values(['videoname', 'ct']).drop_duplicates('videoname').sort_values('viewcnt', ascending=False)[['videoname', 'viewcnt', 'ct']].reset_index(drop=True)


#18. 싫어요/좋아요 비율을 ratio 컬럼으로 만들고 videoname, ratio로 구성된 데이터 프레임을 ratio를 오름차수 정렬
target = video.sort_values('ct').drop_duplicates('videoname',keep='last')
target['ratio'] =target['dislikecnt'] / target['likecnt']
target.sort_values('ratio')[['videoname','ratio']].reset_index(drop=True)


#19. 2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량
start = pd.to_datetime("2021-11-01 00:00:00")
end = pd.to_datetime("2021-11-01 15:00:00")

target = video.loc[(video["ct"] >= start) & (video['ct'] <= end)].reset_index(drop=True)

def check(x):
    result = max(x) - min(x)
    return result

target[['videoname','viewcnt']].groupby("videoname").agg(check)

#20. 중복되는 각 데이터의 시간대와 videoname
video[video.index.isin(set(video.index)-set(video.drop_duplicates().index))][['videoname','ct']]

#3. 월드컵 출전선수 골기록 데이터

In [ ]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv')

#21. 주어진 전체 기간의 각 나라별 골득점수 상위 5개 국가와 그 득점수를 데이터프레임형태로 출력
df.groupby('Country').sum().sort_values('Goals', ascending=False).head(5)


#22. 주어진 전체 기간동안 골득점을 한 선수가 가장 많은 나라 상위 5개 국가와 그 선수 숫자
df.groupby('Country').size().sort_values(ascending=False).head(5)


#23. 년도 표기가 4자리 숫자로 안된 케이스가 존재한다. 해당 건은 몇건인지 출력
df['yearList'] = df.Years.str.split('-')

def checkFour(x):
    for value in x:
        if len(str(value)) != 4:
            return False
        
    return True
    
df['check'] = df['yearList'].apply(checkFour)
len(df[df.check ==False])


#24. 23에서 발생한 예외 케이스를 제외한 데이터프레임을 df2라고 정의하고 데이터의 행의 숫자 출력
df2 = df[df.check ==True].reset_index(drop=True)
df2.shape[0]


#25. 월드컵 출전횟수를 나타내는 ‘LenCup’ 컬럼을 추가하고 4회 출전한 선수의 숫자를 구하여라
df2['LenCup'] = df2.yearList.str.len()
len(df2.loc[df2.LenCup == 4])           #df2['LenCup'].value_counts()[4]


#26. Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들의 숫자를 구하여라
len(df2.loc[(df2.Country == 'Yugoslavia') & (df2.LenCup == 2)])


#27. 2002년도에 출전한 전체 선수는 몇명
len(df2.loc[df2.Years.str.contains('2002')])


#28. 이름에 ‘carlos’ 단어가 들어가는 선수의 숫자는 몇 명인가? (대, 소문자 구분 x)
#★str.lower(): 소문자로 바꿔주기
len(df2.loc[df2.Player.str.lower().str.contains('carlos')])


#29. 월드컵 출전 횟수가 1회뿐인 선수들 중에서 가장 많은 득점을 올렸던 선수
df2.loc[df2.LenCup == 1].sort_values('Goals', ascending=False).Player.iloc[0]


#30. 월드컵 출전횟수가 1회 뿐인 선수들이 가장 많은 국가
df2.loc[df2.LenCup == 1].Country.value_counts().index[0]

'Brazil'

#4. 서울시 따릉이 이용정보 데이터

In [ ]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')

#31. 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜를 출력
result = df.대여일자.value_counts().sort_index().to_frame()
result.loc[result.대여일자 == result.대여일자.max()].index[0]


#32. 요일을‘day_name’컬럼에 추가하고, 이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력
df.대여일자 = pd.to_datetime(df.대여일자)
df['day_name'] = df.대여일자.dt.day_name()
df.day_name.value_counts().to_frame()


#33. 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력
result = df.groupby(['day_name', '대여소번호']).size().to_frame('size').sort_values(['day_name', 'size'], ascending=False).reset_index()
result.drop_duplicates('day_name',keep='first').reset_index(drop=True)


#34. 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대? 일일권의 경우 일일권과 일일권(비회원)을 모두 포함.
daily = df[df.대여구분코드.isin(['일일권','일일권(비회원)'])].연령대코드.value_counts().sort_index()
total = df.연령대코드.value_counts().sort_index()
ratio = (daily /total).sort_values(ascending=False)
ratio.index[0]


#35. 연령대별 평균 이동거리
df[['연령대코드', '이동거리']].groupby(['연령대코드']).mean()


#36. 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출. 최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 
#★round(data,3) : 소수 몇째자리까지 나타낸 것.
df_20 = df.loc[df.연령대코드 == '20대']
df_20_dis = df_20.loc[df_20.이동거리 >= df_20.이동거리.mean()]
result = df_20_dis.sort_values(['대여일자', '대여소번호'], ascending=True).reset_index(drop=True).iloc[:200].탄소량.astype('float')
round(result.sum()/len(result), 3)


#37. 6월 7일, 10대 이용건수 중앙값은?
#★df.대여일자 == pd.to_datetime('2021-06-07')
df['대여일자'] = pd.to_datetime(df['대여일자'])
df.loc[(df.연령대코드 == '~10대') & (df.대여일자 == pd.to_datetime('2021-06-07'))].이용건수.median()


#38. 평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후 
#각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수
data = df.loc[(df.day_name.isin(['Monday', 'Thuesday', 'Wednesday', 'Thursday', 'Friday']) & (df.대여시간.isin([6, 7, 8])))].groupby(['대여시간', '대여소번호']).size().to_frame('이용 횟수')
data.sort_values(['대여시간','이용 횟수'], ascending=False).groupby('대여시간').head(3)


#39. 이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값
df.loc[(df.이동거리 >= df.이동거리.mean())].이동거리.std()


#40. 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값
df['sex'] = df['성별'].map(lambda x: '남' if x in ['M','m'] else '여')
df[['sex','이동거리']].groupby('sex').mean()

,이동거리
sex,
남,3209.110871
여,3468.575025


#5. 전세계 행복도 지표 데이터

In [ ]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv',encoding='utf-8')

#51. 2018, 2019년도의 행복랭킹 10위를 차지한 나라의 행복점수의 평균
df.loc[df.행복랭킹 == 10].점수.mean()


#52. 각년도의 행복랭킹 50위이내의 나라들의 각각의 행복점수 평균
df.loc[df.행복랭킹 <= 50][['년도','점수']].groupby(['년도']).mean()


#53. 2018년도 데이터들만 추출하여 행복점수와 부패에 대한 인식에 대한 상관계수
df.loc[df.년도 == 2018][['점수','부패에 대한인식']].corr().iloc[0,1]


#54. 2018년도와 2019년도의 행복랭킹이 변화하지 않은 나라명의 수
d1 = df.loc[df.년도==2018][['나라명','행복랭킹']].reset_index(drop=True)
d2 = df.loc[df.년도==2019][['나라명','행복랭킹']].reset_index(drop=True)
len(d1.loc[(d1.나라명 == d2.나라명)].loc[(d1.행복랭킹 == d2.행복랭킹)])


#55. 2019년도 데이터들만 상관계수를 구하고 내림차순, 상위 5개. 컬럼명은 v1,v2,corr으로 표시
data = df.loc[(df.년도 == 2019)].corr().unstack().to_frame().reset_index().dropna()
result = data[data[0] != 1].sort_values(0, ascending=False).drop_duplicates(0)
result.columns = ['v1', 'v2', 'corr']


#56. 상대 GDP 평균 이상의 나라들과 평균 이하의 나라들의 행복점수 평균을 각각 구하고 그 차이값을 출력하라
a = df[df.년도 == 2019]
a_up = a[a.상대GDP >= a.상대GDP.mean()].점수.mean()
a_row =a[a.상대GDP <= a.상대GDP.mean()].점수.mean()
diff = a_up - a_row

#57. 각년도의 부패에 대한인식을 내림차순 정렬했을때 상위 20개 국가의 부패에 대한인식의 평균
df.sort_values(['년도','부패에 대한인식'],ascending=False).groupby('년도').head(20).groupby(['년도']).mean()[['부패에 대한인식']]


#58. 각 년도별 하위 행복점수의 하위 5개 국가의 평균 행복점수를 구하여라
df.groupby('년도').tail(5).groupby('년도').mean()[['점수']]


#59. 2018년도 행복랭킹 50위 이내에 포함됐다가 2019년 50위 밖으로 밀려난 국가의 숫자
data = df[(df.년도 == 2018) & (df.행복랭킹 <= 50)].나라명
data2 = df[(df.년도 == 2019) & (df.행복랭킹 > 50)].나라명
data.isin(data2).value_counts()[1]


#60. 2018년,2019년 모두 기록이 있는 나라들 중 년도별 행복점수가 가장 증가한 나라와 그 증가 수치
count = df.나라명.value_counts() 
target = count[count>=2].index

df2 =df.copy()
multiple = df2[df2.나라명.isin(target)].reset_index(drop=True)
multiple.loc[multiple['년도']==2018,'점수'] = multiple[multiple.년도 ==2018]['점수'].values * (-1)
multiple.groupby('나라명').sum()['점수'].sort_values().to_frame().iloc[-1]

점수    0.87
Name: Burundi, dtype: float64

#5. 지역구 에너지 소비량 데이터

In [ ]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/consum/Tetuan%20City%20power%20consumption.csv')

#61. DateTime컬럼을 통해 각 월별로 몇개의 데이터가 있는지 데이터 프레임으로 구하여라
df['DateTime'] = pd.to_datetime(df['DateTime'])
df['DateTime'].dt.month.value_counts().sort_index().to_frame()


#62. 3월달의 각 시간대별 온도의 평균들 중 가장 낮은 시간대의 온도
target = df[df.DateTime.dt.month ==3]
target.groupby(target.DateTime.dt.hour)['Temperature'].mean().sort_values().values[0]


#63. 3월달의 각 시간대별 온도의 평균들 중 가장 높은 시간대의 온도를 출력
target = df[df.DateTime.dt.month ==3]
target.groupby(target.DateTime.dt.hour)['Temperature'].mean().sort_values().values[-1]


#64. Zone 1 Power Consumption 컬럼의 value값의 크기가 Zone 2 Power Consumption 컬럼의 value값의 크기보다 큰 데이터들의 Humidity의 평균
df[df['Zone 1 Power Consumption'] > df['Zone 2  Power Consumption']].Humidity.mean()


#65. 각 zone의 에너지 소비량의 상관관계를 구해서 데이터 프레임으로 표기
df.iloc[:,-3:].corr()


#66. Temperature의 값이 10미만의 경우 A, 10이상 20미만의 경우 B,20이상 30미만의 경우 C, 그 외의 경우 D라고 할때 각 단계의 데이터 숫자
def split_data(x):
    if x<10:
        return "A"
    elif x<20:
        return 'B'
    elif x<30:
        return 'C'
    else:
        return 'D'
    
df['sp'] = df.Temperature.map(split_data)
df['sp'].value_counts()


#67. 6월 데이터중 12시의 Temperature의 표준편차
df[(df.DateTime.dt.month ==6) & (df.DateTime.dt.hour ==12)].Temperature.std()


#68. 6월 데이터중 12시의 Temperature의 분산
df[(df.DateTime.dt.month ==6) & (df.DateTime.dt.hour ==12)].Temperature.var()


#69. Temperature의 평균이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값
df[df.Temperature >= df.Temperature.mean()].sort_values('Temperature').Humidity.values[3]


#70. Temperature의 중간값 이상의 Temperature의 값을 가지는 데이터를Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값
df[df.Temperature >= df.Temperature.median()].sort_values('Temperature').Humidity.values[3]

80.3

#포켓몬 정보 데이터

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')


#71. Legendary 컬럼은 전설포켓몬 유무를 나타낸다.전설포켓몬과 그렇지 않은 포켓몬들의 HP평균의 차이
df[df.Legendary == True].HP.mean() - df[df.Legendary == False].HP.mean()


#72. Type 1은 주속성 Type 2 는 부속성을 나타낸다. 가장 많은 부속성 종류는?
df['Type 2'].value_counts().index[0]


#73. 가장 많은 Type 1 의 종의 평균 Attack 을 평균 Defense로 나눈값은?
data = df[df['Type 1'] == df['Type 1'].value_counts().index[0]]
data.Attack.mean() / data.Defense.mean()


#74. 포켓몬 세대(Generation) 중 가장많은 Legendary를 보유한 세대는 몇세대?
df[df.Legendary == True].Generation.value_counts('Legendary').index[0]


#★75. ‘HP’, ‘Attack’, ‘Defense’, ‘Sp. Atk’, ‘Sp. Def’, ‘Speed’ 간의 상관 계수중 가장 절댓값이 큰 두 변수와 그 값
target = df[[ 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].corr().unstack().reset_index().rename(columns={0: "corr"})
target[target['corr']!=1].sort_values('corr',ascending=False).iloc[0]


#76. 각 Generation의 Attack으로 오름차순 정렬시 상위 3개 데이터들(18개)의 Attack의 전체 평균
df.sort_values(['Generation', 'Attack']).groupby(['Generation']).head(3).Attack.mean()


#77. 각 Generation의 Attack으로 내림차순 정렬시 상위 5개 데이터들(30개)의 Attack의 전체 평균을 구하여라
df.sort_values(['Generation', 'Attack'], ascending=False).groupby(['Generation']).head(5).Attack.mean()


#78. 가장 흔하게 발견되는 (Type1 , Type2) 의 쌍은 무엇인가?
df[['Type 1','Type 2']].value_counts().head(1)


#79. 한번씩만 존재하는 (Type1 , Type2)의 쌍의 갯수는 몇개인가?
target = df[['Type 1','Type 2']].value_counts()
len(target[target==1])


#★80. 한번씩만 존재하는 (Type1 , Type2)의 쌍을 각 세대(Generation)은 각각 몇개씩 가지고 있는가?
target = df[['Type 1','Type 2']].value_counts()
target2 =target[target==1].reset_index()
lst = []

for value in target2.values:
  t1 = value[0]
  t2 = value[1]
  sp = df[(df['Type 1'] == t1) & (df['Type 2'] == t2)]
  lst.append(sp)

pd.concat(lst).reset_index(drop=True).Generation.value_counts().sort_index()

1     1
2     4
3     5
4    13
5     7
6     9
Name: Generation, dtype: int64

#8. 대한민국 체력장 데이터

In [ ]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/body/body.csv')


#81. 전체데이터의 수축기혈압(최고) - 이완기혈압(최저)의 평균
(df['수축기혈압(최고) : mmHg'] - df['이완기혈압(최저) : mmHg']).mean()


#82. 50~59세 신장평균
df.loc[(50 <= df.측정나이) & (df.측정나이 <= 59)]['신장 : cm'].mean()


#83. 연령대(20~29 : 20대 …)별 인원수를 구하여라
df['연령대'] = df.측정나이 // 10 * 10
df['연령대'].value_counts()


#84. 연령대 (20~29 : 20대 …) 별 등급의 숫자를 데이터 프레임으로 표현하라
df.groupby(['연령대','등급'],as_index=False).size()


#85. 남성 중 A등급과 D등급의 체지방률 평균의 차이(큰 값에서 작은 값의 차)
df[(df.측정회원성별 == 'M') & (df.등급 == 'D')]['체지방율 : %'].mean() - df[(df.측정회원성별 == 'M') & (df.등급 == 'A')]['체지방율 : %'].mean()


#86. 여성 중 A등급과 D등급의 체중의 평균의 차이(큰 값에서 작은 값의 차)를 구하여라
abs(df[(df.측정회원성별 =='F') &(df.등급 =='A')].iloc[:,3].mean() -df[(df.측정회원성별 =='F') &(df.등급 =='D')].iloc[:,3].mean())


#87. 데이터의 bmi(몸무게/키제곱)를 구한 새로운 컬럼을 만들고 남성의 bmi 평균을 구하여라
df['BMI'] = df['체중 : kg'] / (df['신장 : cm']/100)**2
df[df.측정회원성별 =='M'].BMI.mean()


#88. bmi보다 체지방율이 높은 사람들의 체중평균
df[df['BMI'] < df['체지방율 : %']]['체중 : kg'].mean()


#89. 남성과 여성의 악력 평균의 차이를 구하여라
target= df.groupby('측정회원성별')['악력D : kg'].mean()
target.M - target.F


#90.남성과 여성의 교차윗몸일으키기 횟수의 평균의 차이를 구하여라
df.groupby('측정회원성별')['교차윗몸일으키기 : 회'].mean()
target.M - target.F

17.55954185047465

#9. 기온 강수량 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/weather/weather2.csv")


#91. 여름철(6월,7월,8월) 이화동이 수영동보다 높은 기온을 가진 시간대는 몇개
df['time'] = pd.to_datetime(df['time'])
summer = df.loc[df.time.dt.month.isin([6, 7, 8])]
len(summer.loc[summer['이화동기온'] > summer['수영동기온']])


#★92. 이화동과 수영동의 최대강수량의 시간대를 각각 구하여라
df.loc[df['이화동강수']==df['이화동강수'].max()].time.values
df.loc[df['수영동강수']==df['수영동강수'].max()].time.values

array(['2020-07-23T12:00:00.000000000'], dtype='datetime64[ns]')

#10. 서비스 이탈예측 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/train.csv")


#93. 남성 이탈(Exited)이 가장 많은 국가(Geography)는 어디이고 이탈 인원은 몇명인가?
df.loc[df.Gender == 'Male'].groupby(['Geography']).sum().Exited.sort_values(ascending=False).head(1)


#94. 카드를 소유(HasCrCard ==1)하고 있으면서 활성멤버(IsActiveMember ==1) 인 고객들의 평균 나이를 소숫점이하 4자리까지
data = df.loc[(df.HasCrCard == 1) & (df.IsActiveMember ==1)].Age.mean()
round(data, 4)


#95. Balance 값이 중간값 이상을 가지는 고객들의 CreditScore의 표준편차를 소숫점이하 3자리까지
answer = df.loc[df.Balance >= df.Balance.median()].CreditScore.std()
round(answer,3)

97.295

#11. 성인 건강검진 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv")


#96. 수축기혈압과 이완기 혈압기 수치의 차이를 새로운 컬럼(‘혈압차’) 으로 생성하고, 연령대 코드별 각 그룹 중 ‘혈압차’ 의 분산이 5번째로 큰 연령대 코드
df['혈압차'] = df.수축기혈압 - df.이완기혈압
df.groupby(['연령대코드(5세단위)']).혈압차.var().sort_values().index[-5]


#97. 비만도를 나타내는 지표인 WHtR는 허리둘레 / 키로 표현한다. 일반적으로 0.58이상이면 비만으로 분류한다. 데이터중 WHtR 지표상 비만인 인원의 남/여 비율을 구하여라
df['WHtR'] = df['허리둘레'] / df['신장(5Cm단위)']
data = df.loc[df.WHtR >= 0.58].성별코드.value_counts()
data['M']/data['F']

1.1693877551020408

#12. 자동차 보험가입 예측데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/train.csv")
df.head()

#98. Vehicle_Age 값이 2년 이상인 사람들만 필터링, Annual_Premium 값이 전체 데이터의 중간값 이상인 사람들을 찾고, 그들의 Vintage값의 평균
df.loc[(df.Vehicle_Age == '> 2 Years') & (df.Annual_Premium >= df.Annual_Premium.median())].Vintage.mean()


#★99. vehicle_age에 따른 각 성별(gender)그룹의 Annual_Premium값의 평균
data = df.groupby(['Gender','Vehicle_Age'], as_index=False)['Annual_Premium'].mean()
data.pivot(index='Vehicle_Age', columns='Gender', values='Annual_Premium')

Gender,Female,Male
Vehicle_Age,,
1-2 Year,30762.245001,30413.088469
< 1 Year,29972.286702,30310.982212
> 2 Years,36108.366374,35303.870627


#13. 핸드폰 가격 예측 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/mobile/train.csv")


#100. price_range 의 각 value를 그룹핑하여 각 그룹의 n_cores 의 빈도가 가장높은 value와 그 빈도수
df[['price_range','n_cores']].groupby(['price_range','n_cores']).size().sort_values(0).groupby(level=0).tail(1)


#★101. price_range 값이 3인 그룹에서 상관관계가 2번째로 높은 두 컬럼과 그 상관계수
data = df.loc[df.price_range == 3].corr().unstack().sort_values(ascending=False)
data.loc[data != 1].reset_index().iloc[1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of Series.sort_values will be keyword-only
  


level_0          pc
level_1          fc
0          0.635166
Name: 1, dtype: object

#14. 비행탑승 경험 만족도 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/train.csv")

#102. Arrival Delay in Minutes 컬럼이 결측치인 데이터들 중 ‘neutral or dissatisfied’ 보다 ‘satisfied’의 수가 더 높은 Class는 어디 인가?
#방법1
df.loc[df['Arrival Delay in Minutes'].isnull()].loc[(df['satisfaction']=='neutral or dissatisfied') < (df['satisfaction']=='satisfied')].Class.unique()

#방법2
answer = df.loc[df['Arrival Delay in Minutes'].isnull()].groupby(['Class','satisfaction'],as_index=False).size().pivot(index='Class',columns='satisfaction')
answer[answer['size']['neutral or dissatisfied'] < answer['size']['satisfied']]

size          
satisfaction neutral or dissatisfied satisfied
Class                                         
Business                          36        76

#15. 수질 음용성 여부 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/train.csv")

#103. ph값은 상당히 많은 결측치를 포함한다. 결측치를 제외한 나머지 데이터들 중 사분위값 기준 하위 25%의 값들의 평균값은?
target = df['ph'].dropna()
target.loc[target <= target.quantile(0.25)].mean()


5.057093462441731

#16. 의료 비용 예측 데이터

In [ ]:
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv")

#104. 흡연자와 비흡연자 각각 charges의 상위 10% 그룹의 평균의 차이는?
high = train.loc[train.smoker =='yes'].charges.quantile(0.9)
high2 = train.loc[train.smoker =='no'].charges.quantile(0.9)
mean_yes = train.loc[(train.smoker =='yes') &(train.charges >=high)].charges.mean()
mean_no = train.loc[(train.smoker =='no') &(train.charges >=high2)].charges.mean()
mean_yes - mean_no

29297.954548156144

#17. 킹카운티 주거지 가격예측문제 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice//train.csv")
df.head()

#105. bedrooms 의 빈도가 가장 높은 값을 가지는 데이터들의 price의 상위 10%와 하위 10%값의 차이를 구하여라
df.loc[df.bedrooms ==df.bedrooms.value_counts().index[0]].price.quantile(0.9) - df.loc[df.bedrooms ==df.bedrooms.value_counts().index[0]].price.quantile(0.1)


505500.0

#18. 대학원 입학 가능성 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv")


#★106. Serial No. 컬럼을 제외하고 ‘Chance of Admit’을 종속변수, 나머지 변수를 독립변수라 할때, 랜덤포레스트를 통해 회귀 예측을 할 떄 변수중요도 값을 출력하라 (시드값에 따라 순서는 달라질수 있음)
from sklearn.ensemble import RandomForestRegressor

df_t = df.drop([df.columns[0]],axis=1)
x = df_t.drop([df.columns[-1]],axis=1)
y = df_t[df.columns[-1]]

ml = RandomForestRegressor()

ml.fit(x,y)

result=pd.DataFrame({'importance':ml.feature_importances_},x.columns).sort_values('importance',ascending=False)
display(result)

,importance
CGPA,0.799428
GRE Score,0.104978
TOEFL Score,0.028300
LOR,0.023710
SOP,0.022860
University Rating,0.013003
Research,0.007721


#19. 레드 와인 퀄리티 예측 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/train.csv")

#107. quality 값이 3인 그룹과 8인 데이터그룹의 각 컬럼별 독립변수의 표준편차 값의 차이를 구할때 그값이 가장 큰 컬럼명을 구하여라
abs(df.loc[df.quality == 3].std() - df.loc[df.quality == 8].std()).sort_values(ascending=False).index[0]

'total sulfur dioxide'

#20. 약물분류 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/train.csv")

#108. 남성들의 연령대별 (10살씩 구분 0~9세 10~19세 …) Na_to_K값의 평균값을 구해서 데이터 프레임으로 표현하여라

df = df.loc[df.Sex == 'M']
df['Age2'] = df.Age // 10 * 10

df.groupby('Age2').Na_to_K.mean().to_frame()

,Na_to_K
Age2,
10,13.627000
20,18.260769
30,15.244143
40,13.897273
50,14.811273
60,14.284308
70,11.363857


#21. 사기회사 분류 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/train.csv")
df.head()

#109. 데이터의 Risk 값에 따른 score_a와 score_b의 평균값을 구하여라
df.groupby(['Risk'])[['Score_A','Score_B']].mean()


,Score_A,Score_B
Risk,,
0,0.262234,0.225532
1,0.490164,0.444262


#22. 센서데이터 동작유형 분류 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/train.csv")
df.head()

#110. pose값에 따른 각 motion컬럼의 중간값의 가장 큰 차이를 보이는 motion컬럼은 어디이며 그값은?
#.T 행열 바꾸기
data = df.groupby('pose').median().T
abs(data[0]-data[1]).sort_values(ascending=False).head(1)

motion_54    4.0
dtype: float64

#23. 현대 차량 가격 분류문제 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/train.csv")

df.head()

#111. 정보(row수)가 가장 많은 상위 3차종의 price값의 각 평균값은?
df.loc[df.model.isin(df.value_counts('model').head(3).index)].groupby('model')[['price']].mean()


,price
model,
I10,7646.137891
I30,11651.821759
Tucson,15805.441373


#24. 당뇨여부판단 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/train.csv")
df.head()

#112. Outcome 값에 따른 각 그룹의 각 컬럼의 평균 차이를 구하여라
#diff() 차이
df.groupby('Outcome').mean().diff().iloc[1,:]


Pregnancies                  1.574159
Glucose                     29.943995
BloodPressure                3.202079
SkinThickness                2.952033
Insulin                     33.398645
BMI                          4.541437
DiabetesPedigreeFunction     0.143646
Age                          5.866939
Name: 1, dtype: float64

#25. 넷플릭스 주식 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nflx/NFLX.csv")
df.head()

#★113. 매년 5월달의 open가격의 평균값을 데이터 프레임으로 표현
df['Date'] = pd.to_datetime(df['Date'])
target = df.groupby(df['Date'].dt.strftime('%Y-%m')).mean()
target.loc[target.index.str.contains('-05')].Open

Date
2002-05      1.155833
2003-05      1.641497
2004-05      4.261143
2005-05      1.951905
2006-05      4.163571
2007-05      3.159351
2008-05      4.435034
2009-05      5.790571
2010-05     14.417071
2011-05     34.650272
2012-05     10.523247
2013-05     31.936429
2014-05     51.121292
2015-05     85.057429
2016-05     92.705715
2017-05    158.255455
2018-05    329.779541
2019-05    359.664548
2020-05    433.880499
2021-05    496.923996
Name: Open, dtype: float64

#26. nba 선수 능력치 데이터

In [ ]:
pd.set_option('display.max_columns',50)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nba/nba.csv",encoding='latin',sep=';')


#114. Tm 컬럼은 각 팀의 이름을 의미한다. TOR팀의 평균나이를 소수 4째 자리까지 구하여라
round(df[df.Tm == 'TOR'].Age.mean(),4)


#★115. Pos 컬럼은 포지션을 의미한다. 전체 선수 중 최소나이대의 선수들을 필터하고 그들 중 가장 많은 포지션은 무엇인지 확인하라
df[df.Age==df.Age.min()].Pos.value_counts().index[0]


#116. 선수들의 이름은 first_name+ 공백 + last_name으로 이루어져 있다. 가장 많은 first_name은 무엇이며 몇 회 발생하는지 확인하라
df['Player'].str.split().str[0].str.lower().value_counts().head(1)


#117. PTS컬럼은 경기당 평균득점수 이다. 각포지션별로 경기당 평균득점수의 평균을 구하여라
df.groupby('Pos').PTS.mean().sort_values()


#118. G컬럼은 참여한 경기의 숫자이다. 각 팀별로 가장 높은 경기참여 수를 가진 선수들의 경기 참여 숫자의 평균을 구하여라
df.sort_values(['Tm','G']).groupby('Tm').tail(1).G.mean()


#119. Tm의 값이 MIA이며 Pos는 C또는 PF인 선수의 MP값의 평균은?
df[(df.Tm =='MIA') & (df.Pos.isin(['C','PF']))].MP.mean()


#120. 전체 데이터중 G의 평균값의 1.5배 이상인 데이터들만 추출했을때 3P값의 평균은?
df[df.G >= df.G.mean()*1.5]['3P'].mean()


#121. Age의 평균 이상인 그룹과 평균 미만인 그룹간의 G값의 평균의 차이는?
df[df.Age >= df.Age.mean()].G.mean() - df[df.Age < df.Age.mean()].G.mean()


#122. 평균나이가 가장 젊은 팀은 어디인가
df.groupby('Tm')['Age'].mean().sort_values().index[0]


#123. Pos그룹별 평균 MP값을 구하여라
df.groupby('Pos').MP.mean()


Pos
C        16.990000
C-PF     25.350000
PF       17.937500
PF-SF    27.300000
PG       19.547742
PG-SG    37.200000
SF       17.514557
SF-SG    20.340000
SG       18.554271
SG-PG    22.950000
SG-SF    19.620000
Name: MP, dtype: float64